In [1]:
USER = 'laphisboy'

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

MODEL_SAVE_PATH = '/content/gdrive/My Drive/Colab Notebooks/dacon_12/model_save/'

Mounted at /content/gdrive


In [3]:
pip install git+https://github.com/laphisboy/pytorch-image-models.git

  Cloning https://github.com/laphisboy/pytorch-image-models.git to /tmp/pip-req-build-suwt3l81
  Running command git clone -q https://github.com/laphisboy/pytorch-image-models.git /tmp/pip-req-build-suwt3l81
  Created wheel for timm: filename=timm-0.4.2-cp36-none-any.whl size=266018 sha256=e8445d2d21e0ed5b4bc2b81ea8dece88a90703912955638aefced7ce17701e56
  Stored in directory: /tmp/pip-ephem-wheel-cache-k6yv8a86/wheels/61/8c/86/d8837b09a6d24d66496d9612389e2b244f129d1083286ab198
Successfully built timm


In [4]:
from google.colab import output

DATA_ZIP_PATH = '/content/gdrive/My Drive/Colab Notebooks/data/data_2/'

# !cp 파일1 파일2 # 파일1을 파일2로 복사 붙여넣기
!cp "/content/gdrive/My Drive/Colab Notebooks/data/data_2_2.zip" "data_2_2.zip"
# data_2.zip을 현재 디렉터리에 압축해제
!unzip "data_2_2.zip"

Archive:  data_2_2.zip
  inflating: dirty_mnist_2nd.zip     
  inflating: dirty_mnist_2nd_answer.csv  
  inflating: mnist_data.zip          
  inflating: sample_submission.csv   
  inflating: test_dirty_mnist_2nd.zip  


In [5]:
# 현재 디렉터리에 dirty_mnist라는 폴더 생성
#!mkdir "./dirty_mnist_2nd"
#dirty_mnist.zip라는 zip파일을 dirty_mnist라는 폴더에 압축 풀기
#!unzip "dirty_mnist_2nd.zip" -d "./dirty_mnist_2nd/"
# 현재 디렉터리에 test_dirty_mnist라는 폴더 생성
!mkdir "./test_dirty_mnist_2nd"
#test_dirty_mnist.zip라는 zip파일을 test_dirty_mnist라는 폴더에 압축 풀기
!unzip "test_dirty_mnist_2nd.zip" -d "./test_dirty_mnist_2nd/"
# 출력 결과 지우기
output.clear()

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import imutils
import zipfile
import os
from PIL import Image
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.models as models
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from torch.utils.data import DataLoader, Dataset
from google.colab import output

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
#dirty_mnist_answer = pd.read_csv(DIRTY_MNIST_ANSWER_PATH)

In [8]:
dirty_mnist_answer = pd.read_csv('dirty_mnist_2nd_answer.csv')
# dirty_mnist라는 디렉터리 속에 들어있는 파일들의 이름을 
# namelist라는 변수에 저장
#namelist = os.listdir('./dirty_mnist/')

In [9]:
dirty_mnist_answer

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,0,1,1,0,1,0,1,0,0,0,0,1,1,0,0,1,1,0,1,1,0,1,0,0,1,1,1
1,1,1,0,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,0,1,1
2,2,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,1,1,1,0,1,1,1,0
3,3,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,1,1,0,1,1,0,1,1,0,1,0
4,4,0,1,0,1,0,1,0,1,1,0,1,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,0,1,1,0,0,0,0,1,0,0,1,1,0,1,1,0,0,0,0,1,0,0,0,1,1,0
49996,49996,0,1,0,1,0,1,1,1,0,1,0,0,0,0,1,0,0,1,1,1,0,1,0,0,0,1
49997,49997,0,1,0,0,1,1,1,1,0,0,1,1,0,0,0,0,1,0,0,0,0,1,1,1,0,0
49998,49998,0,1,1,1,0,0,1,1,0,1,1,1,1,1,0,0,0,1,1,1,0,0,0,1,0,0


In [10]:


#namelist = os.listdir(DIRTY_MNIST_PATH)

In [11]:
# check on the images

def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.show()

In [12]:
class DatasetMNIST(torch.utils.data.Dataset):
    def __init__(self,
                 dir_path,
                 meta_df,
                 transforms,
                 augmentations=None):
        self.dir_path = dir_path
        self.meta_df = meta_df

        self.transforms = transforms
        self.augmentations = augmentations

    def __len__(self):
        return len(self.meta_df)

    def __getitem__(self,index):

        #image = cv2.imread(self.dir_path +\
        #                   str(self.meta_df.iloc[index,0]).zfill(5) + '.png',
        #                   cv2.IMREAD_COLOR)
        
        image = Image.open(
            self.dir_path+str(self.meta_df.iloc[index,0]).zfill(5) + '.png',
        )

        image = self.transforms(image)

        # image = (image/255).astype('float')[..., np.newaxis]
        # might need to check on why this is necessary

        label = self.meta_df.iloc[index, 1:].values.astype('float')

        return image, torch.FloatTensor(label)

In [14]:
# nn.Module을 상속 받아 MultiLabelResnet를 정의
class MultiLabelAntiEfficientNet(nn.Module):
    def __init__(self):
        super(MultiLabelAntiEfficientNet, self).__init__()
        #self.conv2d = nn.Conv2d(1, 3, 3, stride=1)
        self.model = timm.create_model('tf_efficientnet_b4_ns', pretrained=False)
        self.model.conv_stem = nn.Conv2d(1, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, 10)

    def forward(self, x):
        # resnet의 입력은 [3, N, N]으로
        # 3개의 채널을 갖기 때문에
        # resnet 입력 전에 conv2d를 한 층 추가
        #x = F.relu(self.conv2d(x))

        # resnet18을 추가
        # = F.relu(self.model(x))

        # 마지막 출력에 nn.Linear를 추가
        # multilabel을 예측해야 하기 때문에
        # softmax가 아닌 sigmoid를 적용
        x = self.model(x)
        return x
# 모델 선언


In [15]:
MODEL = 'antiefficient'
AFTERTRAINED = 'afterEMNIST'
path = MODEL_SAVE_PATH


In [16]:
# try ensemble and submit
eval_transform = T.Compose([
                             T.ToTensor(),
                             T.Normalize(0.5350, 0.3007),
])

sample_submission = pd.read_csv("sample_submission.csv")
test_dataset = DatasetMNIST("test_dirty_mnist_2nd/", sample_submission, eval_transform)

batch_size = 128

test_data_loader = DataLoader(
    test_dataset,
    batch_size = batch_size,
    shuffle = False,
    num_workers = 3,
    drop_last = False
)

In [20]:
predictions_list = []

prediction_df = pd.read_csv("sample_submission.csv")



for fold in [1,2,3,5]:
    model = torch.load(f'{path}{USER}_{MODEL}_{AFTERTRAINED}_epoch_18_{fold}.pth')
    model.to(device)

    prediction_array = np.zeros([prediction_df.shape[0], prediction_df.shape[1]-1])

    for idx, (images, _) in enumerate(test_data_loader):
        with torch.no_grad():
            model.eval()
            images = images.to(device)
            probs = model(images)
            probs = probs.cpu().detach().numpy()
            preds = (probs > 0.5)

            batch_index = batch_size * idx
            prediction_array[batch_index: batch_index + images.shape[0], :] = preds.astype(int)
    predictions_list.append(prediction_array[..., np.newaxis])


for fold in [1,2,3,5]:
    model = torch.load(f'{path}{USER}_{MODEL}_{AFTERTRAINED}_epoch_18_{fold}.pth')
    model.to(device)

    prediction_array = np.zeros([prediction_df.shape[0], prediction_df.shape[1]-1])
    for idx, (images, _) in enumerate(test_data_loader):
        with torch.no_grad():
            model.eval()
            images = images.to(device)
            images = TF.rotate(images, 90)
            probs = model(images)
            probs = probs.cpu().detach().numpy()
            preds = (probs > 0.5)

            batch_index = batch_size * idx
            prediction_array[batch_index: batch_index + images.shape[0], :] = preds.astype(int)
    predictions_list.append(prediction_array[..., np.newaxis])



for fold in [1,2,3,5]:
    model = torch.load(f'{path}{USER}_{MODEL}_{AFTERTRAINED}_epoch_18_{fold}.pth')
    model.to(device)

    prediction_array = np.zeros([prediction_df.shape[0], prediction_df.shape[1]-1])
    for idx, (images, _) in enumerate(test_data_loader):
        with torch.no_grad():
            model.eval()
            images = images.to(device)
            images = TF.rotate(images, 180)
            probs = model(images)
            probs = probs.cpu().detach().numpy()
            preds = (probs > 0.5)

            batch_index = batch_size * idx
            prediction_array[batch_index: batch_index + images.shape[0], :] = preds.astype(int)

    predictions_list.append(prediction_array[..., np.newaxis])

for fold in [1,2,3,5]:
    model = torch.load(f'{path}{USER}_{MODEL}_{AFTERTRAINED}_epoch_18_{fold}.pth')
    model.to(device)

    prediction_array = np.zeros([prediction_df.shape[0], prediction_df.shape[1]-1])
    for idx, (images, _) in enumerate(test_data_loader):
        with torch.no_grad():
            model.eval()
            images = images.to(device)
            images = TF.rotate(images, 270)
            probs = model(images)
            probs = probs.cpu().detach().numpy()
            preds = (probs > 0.5)

            batch_index = batch_size * idx
            prediction_array[batch_index: batch_index + images.shape[0], :] = preds.astype(int)
    predictions_list.append(prediction_array[..., np.newaxis])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:79: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [21]:
predictions_array = np.concatenate(predictions_list, axis=2)
predictions_mean = predictions_array.mean(axis=2)

predictions_mean = (predictions_mean >= 0.5) * 1 # * 1 이 필요하나
print(predictions_mean)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [22]:
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission.iloc[:, 1:] = predictions_mean
sample_submission.to_csv("ensembletta_voteagg.csv", index=False)
sample_submission

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,50000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,50001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,50002,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,50003,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,50004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,54995,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4996,54996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4997,54997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4998,54998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [ ]:
# 0.6315384615	

In [24]:
predictions_list = []

prediction_df = pd.read_csv("sample_submission.csv")



for fold in [1,2,3,5]:
    model = torch.load(f'{path}{USER}_{MODEL}_{AFTERTRAINED}_epoch_18_{fold}.pth')
    model.to(device)

    prediction_array = np.zeros([prediction_df.shape[0], prediction_df.shape[1]-1])

    for idx, (images, _) in enumerate(test_data_loader):
        with torch.no_grad():
            model.eval()
            images = images.to(device)
            probs = model(images)
            probs = F.softmax(probs)
            probs = probs.cpu().detach().numpy()
            preds = probs

            batch_index = batch_size * idx
            prediction_array[batch_index: batch_index + images.shape[0], :] = preds.astype(float)
    predictions_list.append(prediction_array[..., np.newaxis])


for fold in [1,2,3,5]:
    model = torch.load(f'{path}{USER}_{MODEL}_{AFTERTRAINED}_epoch_18_{fold}.pth')
    model.to(device)

    prediction_array = np.zeros([prediction_df.shape[0], prediction_df.shape[1]-1])
    for idx, (images, _) in enumerate(test_data_loader):
        with torch.no_grad():
            model.eval()
            images = images.to(device)
            images = TF.rotate(images, 90)
            probs = model(images)
            probs = F.softmax(probs)
            probs = probs.cpu().detach().numpy()
            preds = probs

            batch_index = batch_size * idx
            prediction_array[batch_index: batch_index + images.shape[0], :] = preds.astype(float)
    predictions_list.append(prediction_array[..., np.newaxis])



for fold in [1,2,3,5]:
    model = torch.load(f'{path}{USER}_{MODEL}_{AFTERTRAINED}_epoch_18_{fold}.pth')
    model.to(device)

    prediction_array = np.zeros([prediction_df.shape[0], prediction_df.shape[1]-1])
    for idx, (images, _) in enumerate(test_data_loader):
        with torch.no_grad():
            model.eval()
            images = images.to(device)
            images = TF.rotate(images, 180)
            probs = model(images)
            probs = F.softmax(probs)
            probs = probs.cpu().detach().numpy()
            preds = probs

            batch_index = batch_size * idx
            prediction_array[batch_index: batch_index + images.shape[0], :] = preds.astype(float)

    predictions_list.append(prediction_array[..., np.newaxis])

for fold in [1,2,3,5]:
    model = torch.load(f'{path}{USER}_{MODEL}_{AFTERTRAINED}_epoch_18_{fold}.pth')
    model.to(device)

    prediction_array = np.zeros([prediction_df.shape[0], prediction_df.shape[1]-1])
    for idx, (images, _) in enumerate(test_data_loader):
        with torch.no_grad():
            model.eval()
            images = images.to(device)
            images = TF.rotate(images, 270)
            probs = model(images)
            probs = F.softmax(probs)
            probs = probs.cpu().detach().numpy()
            preds = probs

            batch_index = batch_size * idx
            prediction_array[batch_index: batch_index + images.shape[0], :] = preds.astype(float)
    predictions_list.append(prediction_array[..., np.newaxis])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:79: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [25]:
predictions_array = np.concatenate(predictions_list, axis=2)
predictions_mean = predictions_array.mean(axis=2)

predictions_mean = (predictions_mean >= 0.5) * 1 # * 1 이 필요하나
print(predictions_mean)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [26]:
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission.iloc[:, 1:] = predictions_mean
sample_submission.to_csv("ensembletta_softagg.csv", index=False)
sample_submission

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,50000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,50001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,50002,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,50003,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,50004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,54995,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4996,54996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4997,54997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4998,54998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
predictions_array

array([[[1.55891459e-02, 3.60923900e-06, 2.06330746e-01, ...,
         1.77475009e-02, 4.21094656e-01, 4.73621003e-05],
        [1.13859176e-06, 1.43541221e-10, 1.31017769e-05, ...,
         2.49970327e-07, 2.65304861e-05, 3.41905015e-09],
        [6.56449876e-04, 9.99894500e-01, 2.60524582e-02, ...,
         4.72475499e-01, 1.71956094e-03, 2.97883762e-05],
        ...,
        [7.00162118e-03, 1.49343232e-05, 4.32736784e-01, ...,
         2.66550668e-02, 4.68804799e-02, 7.35014692e-06],
        [1.09020891e-06, 1.05204501e-09, 4.97546853e-06, ...,
         1.14294744e-06, 5.16369755e-06, 6.54350840e-09],
        [3.91739188e-03, 5.52628705e-07, 3.31372744e-03, ...,
         2.14681821e-03, 5.65930735e-04, 8.61553272e-05]],

       [[1.31567076e-05, 5.32904141e-06, 1.21806424e-05, ...,
         3.21622770e-06, 2.01700677e-05, 1.37051129e-05],
        [5.05447460e-05, 2.95528416e-02, 8.55343067e-04, ...,
         8.91826954e-03, 1.55653281e-04, 1.60828717e-02],
        [1.18601718e-04, 

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Feb  5 14:04:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    55W / 300W |   6573MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!
